In [1]:
import os

# đọc video từ thư mục data/video
video_path = "data/video/"
video_list = os.listdir(video_path)

In [2]:
# thêm vào danh sách video các video lật ngược name+f
new_video_list = video_list.copy()

for video in video_list:
    new_video_list.append(video[:-4] + "f.mp4")

video_list = new_video_list

In [3]:
import cv2
import sys

sys.path.append("../module/")
from init import init_csv
from pose_landmarker import extract_pose_features

for video_name in video_list:
    # Nếu tên video có chữ f ở cuối thì xóa chữ f đi và lật ngược video
    if video_name[-5] == "f":
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name[:-5] + ".mp4"))
    else:
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name))

    # Kiểm tra xem video có được mở thành công không
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()

    # Khởi tạo csv_writer
    csv_writer = init_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Đọc từng frame từ video
    while True:
        ret, frame = cap.read()

        # Kiểm tra nếu có chữ f ở cuối tên video thì lật ngược frame
        if video_name[-5] == "f":
            frame = cv2.flip(frame, 1)

        # Kiểm tra xem có còn frame nào để đọc không
        if not ret:
            break

        # Trích xuất đặc trưng từ frame
        landmarks = extract_pose_features(frame)

        # Nếu trích xuất được đặc trưng thì ghi dữ liệu xuống file csv
        if landmarks is not None:
            # Ghi dữ liệu xuống file csv
            csv_writer.writerow([j for i in landmarks for j in i])

# Đóng cửa sổ hiển thị và giải phóng tài nguyên
cv2.destroyAllWindows()

In [4]:
import pandas as pd
import csv

# Khởi tạo csv_writer
csv_file = open("data/data.csv", "w", newline="")  # Tạo file csv
csv_writer = csv.writer(csv_file)  # Khởi tạo csv_writer

# Khởi tạo header cho file csv
header = [f"{i}" for i in range(0, 1170)] + ["label"]
csv_writer.writerow(header)

for video_name in video_list:
    # Đọc file csv
    csv_reader = pd.read_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Lấy 28 dòng đầu tiên của file csv và chuyển nó thành mảng 1 chiều
    list = [j for i in csv_reader.head(30).values for j in i]

    # Nếu tên video có số 0 đầu thì thêm số 0 và cuối list
    if video_name.split(".")[0][0] == "0":
        list.append(0)
    else:
        list.append(1)

    # Ghi dữ liệu xuống file csv
    csv_writer.writerow(list)

# Đóng file csv
csv_file.close()

In [5]:
# Đọc file data.csv
csv_reader = pd.read_csv("data/data.csv")

# In ra 5 dòng đầu tiên của file csv
csv_reader

,0,1,2,3,4,5,6,7,8,9,...,1161,1162,1163,1164,1165,1166,1167,1168,1169,label
0,0.479832,0.226308,-0.191834,0.522566,0.299079,0.029988,0.441129,0.297831,0.039410,0.520958,...,0.439730,0.641632,-0.022814,0.516509,0.778162,0.203044,0.433096,0.783251,0.180274,0
1,0.473642,0.218424,-0.244884,0.516687,0.285470,0.002815,0.432286,0.283625,0.011435,0.555427,...,0.437831,0.638412,-0.023069,0.496939,0.799569,0.106396,0.432793,0.775588,0.152766,0
2,0.492018,0.201748,-0.240479,0.535070,0.287116,0.011310,0.450817,0.278284,0.000268,0.553667,...,0.454501,0.649078,-0.035261,0.495688,0.793971,0.183968,0.462640,0.793851,0.168975,0
3,0.476379,0.211881,-0.186378,0.518921,0.283963,0.019227,0.437149,0.287399,0.036340,0.546644,...,0.430748,0.636678,-0.033758,0.498893,0.795711,0.078808,0.407627,0.780113,0.086856,0
4,0.483701,0.211692,-0.235794,0.526128,0.287072,0.043727,0.442634,0.289683,-0.014100,0.534582,...,0.454870,0.652144,-0.045462,0.502551,0.789684,0.178976,0.462765,0.791440,0.158383,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,0.204523,0.733782,-0.227324,0.257237,0.705036,-0.192469,0.242906,0.747514,0.016314,0.276712,...,0.425989,0.800540,-0.213808,0.416700,0.734804,0.092212,0.401209,0.757304,-0.099815,1
150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.245386,0.838451,0.157875,0.219493,0.812909,0.232301,0.224163,0.810599,0.277634,1
151,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.439368,0.809063,0.109332,0.572464,0.867792,0.028608,0.576406,0.839241,0.166121,1
152,0.137085,0.710333,-0.135152,0.188605,0.586978,-0.135624,0.191248,0.712527,-0.017664,0.207527,...,0.588067,0.805701,-0.046036,0.496806,0.780743,0.146753,0.480820,0.798310,-0.002984,1
